In [20]:
import json
import requests
import pandas as pd
from config import key_dict
from pprint import pprint
api_key = key_dict['oiko_weather']


#load hundred_city_df
hundred_city_df = pd.read_csv (r'Resources\100_countiesdata.csv')
#clean up hundred_city_df
hundred_city_df = hundred_city_df.drop(labels=["county_ascii"],axis=1)
hundred_city_df.rename(columns={"county_x":"county"},inplace=True)
#create weather_df
column_names=['county','state_name','county_fips','state_id','lat','lng','population']
weather_df = pd.DataFrame(columns = column_names)
weather_df, hundred_city_df

(Empty DataFrame
 Columns: [county, state_name, county_fips, state_id, lat, lng, population]
 Index: [],
        FIPS      county  county_fips state_id    state_name      lat  \
 0   31117.0   McPherson        31117       NE      Nebraska  41.5682   
 1   31105.0     Kimball        31105       NE      Nebraska  41.1978   
 2   27013.0  Blue Earth        27013       MN     Minnesota  44.0346   
 3   41015.0       Curry        41015       OR        Oregon  42.4576   
 4   41033.0   Josephine        41033       OR        Oregon  42.3655   
 ..      ...         ...          ...      ...           ...      ...   
 95   8089.0       Otero         8089       CO      Colorado  37.9026   
 96  42031.0     Clarion        42031       PA  Pennsylvania  41.1924   
 97  16041.0    Franklin        16041       ID         Idaho  42.1811   
 98  53031.0   Jefferson        53031       WA    Washington  47.7489   
 99  50023.0  Washington        50023       VT       Vermont  44.2734   
 
          lng  po

In [21]:
### THIS IS THE REAL DEAL API CALL

for index, row in hundred_city_df.iterrows():
    #print(row)
    lat = row['lat']
    lng = row['lng']
    county_fips = row['county_fips']
    county = row['county']
    state_name = row['state_name']
    state_id = row['state_id']
    population = row['population']
    ###---  The Request!!!
    try:
        r = requests.get('https://api.oikolab.com/weather',
                         params={'param': ['temperature','wind_speed','surface_solar_radiation',
                                           'total_cloud_cover', 'total_precipitation',
                                           'relative_humidity'],
                                 'start': '2020-01-21',
                                 'end': '2021-03-17',
                                 'freq':"D",
                                 'lat': lat,
                                 'lon': lng,
                                 'api-key': api_key})                 
        ## store json resonse per call in weather_df
        #print(index, row, r)
        weather_data = json.loads(r.json()['data'])
        #make a temp dataframe to pass the data
        temp_df = pd.DataFrame(
                      index=pd.to_datetime(weather_data['index'],unit='s'),
                      data=weather_data['data'],
                      columns=weather_data['columns'])
        #print(temp_df)
        #clean the temperory dataframe
        temp_df.reset_index(inplace=True)
        temp_df = temp_df.rename(columns={'index':'date'})
        #fill in values 
        #temp_df = temp_df.reindex(columns = temp_df.columns+column_names)
        #print(temp_df)
        temp_df['county']=county
        temp_df['state_name'] = state_name
        temp_df['county_fips']=county_fips
        temp_df['state_id']= state_id
        temp_df['population']=population
        temp_df['lat']=lat
        temp_df['lng']=lng
        #print(temp_df)
        #append the temporar dataframe to weather_df
        weather_df = weather_df.append(
                        other=temp_df,
                        ignore_index=True)
        temp_df.to_csv(f"weather_temp\{county}.csv",index=False)
    except:
        print(f"{county} skipped")
    
weather_df

Eddy skipped
Zavala skipped
Chesterfield skipped
Lincoln skipped
Waushara skipped
Jackson skipped
Harnett skipped
Lincoln skipped
Valencia skipped
Grant skipped
Cumberland skipped
Adams skipped
Luce skipped
Pike skipped
Powder River skipped
Douglas skipped
Shawnee skipped
Geary skipped
Reno skipped
Alger skipped
Sunflower skipped
Otter Tail skipped
Tompkins skipped
Ouray skipped
Calvert skipped
Floyd skipped
Bland skipped
Washington skipped
Madison skipped
Sebastian skipped
Scott skipped
Kenai Peninsula skipped
Jackson skipped
Payette skipped
Raleigh skipped
Catron skipped
Otero skipped
Clarion skipped
Franklin skipped
Jefferson skipped
Washington skipped


,county,state_name,county_fips,state_id,lat,lng,population,date,temperature (degC),wind_speed (m/s),surface_solar_radiation (W/m^2),total_cloud_cover (0-1),total_precipitation (mm of water equivalent),relative_humidity (0-1)
0,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-21,2.46,2.38,101.0,0.84,0.00,0.65
1,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-22,1.88,7.77,98.0,0.70,0.02,0.66
2,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-23,-1.02,2.80,118.0,0.44,0.00,0.78
3,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-24,1.36,1.80,126.0,0.41,0.00,0.67
4,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-25,2.96,3.63,125.0,0.31,0.00,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24893,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-13,20.38,1.06,156.0,0.79,0.00,0.47
24894,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-14,18.43,0.71,269.0,0.23,0.01,0.37
24895,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-15,23.01,3.39,171.0,0.59,0.03,0.67
24896,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-16,24.03,2.11,280.0,0.29,0.04,0.40


In [22]:
import glob
#creates list of csv files
csv_files=glob.glob("weather_temp/*.csv")
file_list=[]

for file in csv_files:
    file_list.append(pd.read_csv (file))
    
df=pd.concat(file_list).reset_index()
df

,index,date,temperature (degC),wind_speed (m/s),surface_solar_radiation (W/m^2),total_cloud_cover (0-1),total_precipitation (mm of water equivalent),relative_humidity (0-1),county,state_name,county_fips,state_id,population,lat,lng
0,0,2020-01-21,1.21,2.79,149,0.33,0.00,0.46,Anderson,South Carolina,45007,SC,198064,34.5193,-82.6378
1,1,2020-01-22,4.41,3.17,107,0.92,0.00,0.44,Anderson,South Carolina,45007,SC,198064,34.5193,-82.6378
2,2,2020-01-23,7.33,2.60,16,1.00,1.08,0.80,Anderson,South Carolina,45007,SC,198064,34.5193,-82.6378
3,3,2020-01-24,7.30,3.24,147,0.29,0.00,0.76,Anderson,South Carolina,45007,SC,198064,34.5193,-82.6378
4,4,2020-01-25,6.08,2.69,159,0.63,0.00,0.59,Anderson,South Carolina,45007,SC,198064,34.5193,-82.6378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24471,417,2021-03-13,6.26,8.94,16,1.00,0.00,0.88,York,Nebraska,31185,NE,13745,40.8727,-97.5971
24472,418,2021-03-14,5.07,5.36,39,1.00,0.00,0.95,York,Nebraska,31185,NE,13745,40.8727,-97.5971
24473,419,2021-03-15,6.16,1.76,78,1.00,0.01,0.87,York,Nebraska,31185,NE,13745,40.8727,-97.5971
24474,420,2021-03-16,2.91,5.74,88,0.99,0.65,0.81,York,Nebraska,31185,NE,13745,40.8727,-97.5971


In [23]:
df.to_csv("Resources\weather_calls.csv",index=False)

In [25]:
weather_df

,county,state_name,county_fips,state_id,lat,lng,population,date,temperature (degC),wind_speed (m/s),surface_solar_radiation (W/m^2),total_cloud_cover (0-1),total_precipitation (mm of water equivalent),relative_humidity (0-1)
0,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-21,2.46,2.38,101.0,0.84,0.00,0.65
1,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-22,1.88,7.77,98.0,0.70,0.02,0.66
2,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-23,-1.02,2.80,118.0,0.44,0.00,0.78
3,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-24,1.36,1.80,126.0,0.41,0.00,0.67
4,McPherson,Nebraska,31117,NE,41.5682,-101.0605,395,2020-01-25,2.96,3.63,125.0,0.31,0.00,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24893,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-13,20.38,1.06,156.0,0.79,0.00,0.47
24894,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-14,18.43,0.71,269.0,0.23,0.01,0.37
24895,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-15,23.01,3.39,171.0,0.59,0.03,0.67
24896,Frio,Texas,48163,TX,28.8678,-99.1082,19871,2021-03-16,24.03,2.11,280.0,0.29,0.04,0.40
